In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load your dataset
df = pd.read_csv(r""C:\Users\jilow\OneDrive\Documents\data\benin-malanville.csv"")

Data profiling

In [ ]:
# Dataset overview: Shape and info
print(f"Shape of dataset: {df.shape}")
print("Columns:", df.columns.tolist())
df.info()# Summary of data types and non-null counts
df.head()

In [ ]:
# Descriptive statistics (numeric)
display(df.describe())

In [ ]:
# Missing‐value report
miss = df.isna().sum()
display(miss[miss > 0])

In [ ]:
# Flag columns with >5% missing values
total = len(df)
for col, n in miss.items():
    if n/total > 0.05:
        print(f"⚠️ {col}: {n} missing ({n/total:.1%})")


Data Cleaning

In [ ]:
import numpy as np
from scipy.stats import zscore

# Convert Timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M')

In [ ]:
# Drop totally empty or irrelevant columns
if 'Comments' in df.columns:
    df.drop(columns=['Comments'], inplace=True)

In [ ]:
# Outlier detection via Z‐score on critical columns
cols = ['GHI','DNI','DHI','ModA','ModB','WS','WSgust']
zs = df[cols].apply(zscore, nan_policy='omit')
outlier_mask = (np.abs(zs) > 3).any(axis=1)
print("Outliers flagged:", outlier_mask.sum())

In [ ]:
# Remove outliers
df_clean = df.loc[~outlier_mask].copy()

In [ ]:
# Impute remaining missing numeric values with median
for c in cols:
    med = df_clean[c].median()
    df_clean[c] = df_clean[c].fillna(med)

In [ ]:
# Final check: no missing & reasonable shape
print("After cleaning shape:", df_clean.shape)
display(df_clean.isna().sum()[df_clean.isna().sum() > 0])

EDA

In [ ]:
# Time series plot
plt.figure(figsize=(12,4))
plt.plot(df_clean['Timestamp'], df_clean['GHI'], alpha=0.3)
plt.title('Benin: GHI Over Time')
plt.xlabel('Timestamp')
plt.ylabel('GHI (W/m²)')
plt.tight_layout()
plt.show()


In [ ]:
# Histogram & Distribution
df_clean[['GHI','DNI','DHI','Tamb']].hist(bins=50, figsize=(12,8))
plt.suptitle('Distributions of Key Variables')
plt.show()

In [ ]:
#cleaning impact on module output
cleaning_means = df_clean.groupby('Cleaning')[['ModA','ModB']].mean().reset_index()
sns.barplot(data=cleaning_means.melt(id_vars='Cleaning'),
            x='Cleaning', y='value', hue='variable')
plt.title('Module Output Pre/Post Cleaning')
plt.ylabel('Mean Output (W/m²)')
plt.show()


In [ ]:
# Correlation heatmap
plt.figure(figsize=(8,6))
sns.heatmap(df_clean[['GHI','DNI','DHI','TModA','TModB','Tamb','RH']].corr(),
            annot=True, cmap='coolwarm')
plt.title('Feature Correlations')
plt.show()

In [ ]:
# Scatter relation
sns.scatterplot(data=df_clean.sample(5000), x='RH', y='GHI', alpha=0.2)
plt.title('Relative Humidity vs GHI')
plt.show()

In [ ]:
# Wind & Direction analysis
# Wind speed distribution
df_clean['WS'].hist(bins=50)
plt.title('Wind Speed Distribution')
plt.xlabel('WS (m/s)')
plt.show()
# Simple wind‐rose style scatter
sns.scatterplot(data=df_clean.sample(2000),
                x='WD', y='WS', hue='WS', palette='viridis', alpha=0.6)
plt.title('Wind Direction vs Speed')
plt.show()

In [ ]:
# Bubble chart: Temperature and humidity
plt.figure(figsize=(8,6))
plt.scatter(df_clean['Tamb'], df_clean['GHI'],
            s=df_clean['RH'], alpha=0.3)
plt.title('GHI vs Temperature (bubble size = RH)')
plt.xlabel('Tamb (°C)')
plt.ylabel('GHI (W/m²)')
plt.show()


In [ ]:
df_clean.to_csv('C:/Users/jilow/solar-challenge-w0/data/benin_clean.csv', index=False)